In [1]:


import numpy as np
from time import time as timer
def checktick():
    M = 200
    timesfound = np.empty((M,))
    for i in range(M):
        t1 = timer() # get timestamp from timer
        t2 = timer() # get timestamp from timer
        while (t2 - t1) < 1e-16: # if zero then we are below clock granularity, retake timing
            t2 = timer() # get timestamp from timer
        t1 = t2 # this is outside the loop
        timesfound[i] = t1 # record the time stamp
    minDelta = 1000000
    Delta = np.diff(timesfound) # it should be cast to int only when needed
    minDelta = Delta.min()
    return minDelta



: 

In [ ]:
checktick()

In [3]:
import numpy as np
from time import perf_counter as timer
def checktick():
    M = 200
    timesfound = np.empty((M,))
    for i in range(M):
        t1 = timer() # get timestamp from timer
        t2 = timer() # get timestamp from timer
        while (t2 - t1) < 1e-16: # if zero then we are below clock granularity, retake timing
            t2 = timer() # get timestamp from timer
        t1 = t2 # this is outside the loop
        timesfound[i] = t1 # record the time stamp
    minDelta = 1000000
    Delta = np.diff(timesfound) # it should be cast to int only when needed
    minDelta = Delta.min()
    return minDelta

In [ ]:
checktick()

In [5]:
import numpy as np
from time import time_ns as timer
def checktick():
    M = 200
    timesfound = np.empty((M,))
    for i in range(M):
        t1 = timer() # get timestamp from timer
        t2 = timer() # get timestamp from timer
        while (t2 - t1) < 1e-16: # if zero then we are below clock granularity, retake timing
            t2 = timer() # get timestamp from timer
        t1 = t2 # this is outside the loop
        timesfound[i] = t1 # record the time stamp
    minDelta = 1000000
    Delta = np.diff(timesfound) # it should be cast to int only when needed
    minDelta = Delta.min()
    return minDelta/pow(10,9)

In [ ]:
checktick()

In [ ]:
"""Julia set generator without optional PIL-based image drawing"""
from timeit import default_timer as time
from functools import wraps

# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193

# decorator to time
def timefn(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = time()
        result = fn(*args, **kwargs)
        t2 = time()
        print(f"@timefn: {fn.__name__} took {t2 - t1} seconds")
        return result
    return measure_time

@timefn
def calc_pure_python(desired_width, max_iterations):
    """Create a list of complex coordinates (zs) and complex parameters (cs),
    build Julia set"""
    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # build a list of coordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our
    # function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    # This sum is expected for a 1000^2 grid with 300 iterations
    # It ensures that our code evolves exactly as we'd intended
    assert sum(output) == 33219980

@timefn
def calculate_z_serial_purepython(maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output

if __name__ == "__main__":
    # Calculate the Julia set using a pure Python solution with
    # reasonable defaults for a laptop
    calc_pure_python(desired_width=1000, max_iterations=300) 

In [ ]:
! python3 -m cProfile -s cumulative JuliaSet.py ##Changed to time perf_counter in file to reduce import clutter in output.
                                                ##Timeit default_counter uses perf_counter so they are identical

: 

In [ ]:
! python3 -m line_profiler JuliaSet.py.lprof

In [ ]:
! python3 -m memory_profiler JuliaSetMemProf.py

In [ ]:
! python3 -m mprof run JuliaSet.py

![title](mprofPlot.png)

In [ ]:
import cProfile
import pstats
# from line_profiler import profile 
# for first task use line_profiler instead of memory_profiler
# from memory_profiler import profile

grid_shape = (640, 640)

def evolve(grid, dt, D=1.0):
    xmax, ymax = grid_shape
    new_grid = [[0.0] * ymax for x in range(xmax)]
    for i in range(xmax):
        for j in range(ymax):
            grid_xx = (
                grid[(i + 1) % xmax][j] + grid[(i - 1) % xmax][j] - 2.0 * grid[i][j]
            )
            grid_yy = (
                grid[i][(j + 1) % ymax] + grid[i][(j - 1) % ymax] - 2.0 * grid[i][j]
            )
            new_grid[i][j] = grid[i][j] + D * (grid_xx + grid_yy) * dt
    return new_grid

# @profile
def run_experiment(num_iterations):
    # Setting up initial conditions 
    xmax, ymax = grid_shape
    grid = [[0.0] * ymax for x in range(xmax)]

    # These initial conditions are simulating a drop of dye in the middle of our
    # simulated region
    block_low = int(grid_shape[0] * 0.4)
    block_high = int(grid_shape[0] * 0.5)
    for i in range(block_low, block_high):
        for j in range(block_low, block_high):
            grid[i][j] = 0.005

    # Evolve the initial conditions
    for i in range(num_iterations):
        grid = evolve(grid, 0.1)

run_experiment(100)


    363 function calls (362 primitive calls) in 8.595 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000    8.594    8.594 {built-in method builtins.exec}
        1    0.003    0.003    8.594    8.594 upp2.py:1(<module>)
        1    0.236    0.236    8.591    8.591 upp2.py:24(run_experiment)
        .
        .
        .
        the rest is 0

In [ ]:
# import cProfile
# import pstats
from line_profiler import profile 
# for first task use line_profiler instead of memory_profiler
# from memory_profiler import profile

grid_shape = (640, 640)

def evolve(grid, dt, D=1.0):
    xmax, ymax = grid_shape
    new_grid = [[0.0] * ymax for x in range(xmax)]
    for i in range(xmax):
        for j in range(ymax):
            grid_xx = (
                grid[(i + 1) % xmax][j] + grid[(i - 1) % xmax][j] - 2.0 * grid[i][j]
            )
            grid_yy = (
                grid[i][(j + 1) % ymax] + grid[i][(j - 1) % ymax] - 2.0 * grid[i][j]
            )
            new_grid[i][j] = grid[i][j] + D * (grid_xx + grid_yy) * dt
    return new_grid

@profile
def run_experiment(num_iterations):
    # Setting up initial conditions 
    xmax, ymax = grid_shape
    grid = [[0.0] * ymax for x in range(xmax)]

    # These initial conditions are simulating a drop of dye in the middle of our
    # simulated region
    block_low = int(grid_shape[0] * 0.4)
    block_high = int(grid_shape[0] * 0.5)
    for i in range(block_low, block_high):
        for j in range(block_low, block_high):
            grid[i][j] = 0.005

    # Evolve the initial conditions
    for i in range(num_iterations):
        grid = evolve(grid, 0.1)
        
# run_experiment(100)
# to test cprofile run the cod(remove @profile):python3 -m cProfile -s cumulative upp2.py
# profiler = cProfile.Profile()
# profiler.run('run_experiment(100)')
# profiler.dump_stats('profile_results.prof')

run_experiment(100) # python3 -m kernprof -l upp2.py  show results: python3 -m line_profiler -rmt "upp2.py.lprof" show
# python3 ...
# snakeviz profile_results.prof

Timer unit: 1e-06 s

Total time: 32.255 s
File: upp2.py
Function: run_experiment at line 23

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    23                                           @profile
    24                                           def run_experiment(num_iterations):
    25                                               # Setting up initial conditions 
    26         1          0.0      0.0      0.0      xmax, ymax = grid_shape
    27         1        602.0    602.0      0.0      grid = [[0.0] * ymax for x in range(xmax)]
    28                                           
    29                                               # These initial conditions are simulating a drop of dye in the middle of our
    30                                               # simulated region
    31         1          3.0      3.0      0.0      block_low = int(grid_shape[0] * 0.4)
    32         1          0.0      0.0      0.0      block_high = int(grid_shape[0] * 0.5)
    33        65          9.0      0.1      0.0      for i in range(block_low, block_high):
    34      4160        439.0      0.1      0.0          for j in range(block_low, block_high):
    35      4096        505.0      0.1      0.0              grid[i][j] = 0.005
    36                                           
    37                                               # Evolve the initial conditions
    38       101        119.0      1.2      0.0      for i in range(num_iterations):
    39       100   32253355.0 322533.5    100.0          grid = evolve(grid, 0.1)

 32.26 seconds - upp2.py:23 - run_experiment

![Memory Plot](./memplot.png)

![Snakeviz Visualization](./snakevizim.png)